# Intermediate Machine Learning with scikit-learn

Sometimes the features you have available in your initial data have little predictive strength when used in the most straightforward way.  This might be true almost regardless of choice of model class and hyperparameters.  And yet it might also be true that there are synthetic features latent in the data that are highly predictive, but that have to be *engineered* (mechanically, rather than sample-wise modification) to produce powerful features.

At the same time, a highly dimension model—whether of high dimension because of the initial data collection or because of creation of extra synthetic features—may lend itself less well to modeling techniques.  In these cases, it can be more computationally tractable, as well as more predictive, to work with a subset of all available features.

In [ ]:
# Some libraries tend to be in flux for their dependency versions
import warnings
warnings.simplefilter("ignore")

# We utilize some mglearn demo functions
import src.mglearn as mglearn

### A synthetic example

Let us look at an artificial example where raw features of a dataset of absolutely no value, but it is possible to derive good predictions by creating syntheric features out of them.  Obviously, real world data will not be as neat as that, but it is useful to express the concept.

At first brush the loaded data seems fairly noisy without an obvious pattern.

In [ ]:
import pandas as pd
linf = pd.read_csv('data/linear_failure.csv', index_col=0)
linf.head()

In [ ]:
# The features seem uncorrelated, and no univariate correlation with target
linf.corr()

In [ ]:
# Distribution of features and target looks roughly Gaussian
linf.hist(figsize=(12,6));

In [ ]:
# No obvious trends in the data as sequences
%matplotlib inline
linf.plot(figsize=(12,6), style='.');

We might hope to identify a relationship between features and target using a linear regression such as this:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = linf.drop('TARGET', axis=1)
y = linf['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y)
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

Slightly different linear models do equally poorly in detecting any relationship between the features and the target.  Notice that the metric used here is $R^2$ score rather than e.g. explained variance or mean absolute error (or others).


In [ ]:
from sklearn.linear_model import Lasso, Ridge
lasso, ridge = Lasso(), Ridge()
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)

lasso.score(X_test, y_test), ridge.score(X_test, y_test)

### Adding a feature

Let us try creating a new feature that is entirely based on existing features.

In [ ]:
linf['f1xf2'] = linf.feature_1 * linf.feature_2
linf.head()

In [ ]:
X = linf.drop('TARGET', axis=1)
y = linf['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y)
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

The information we need was "latent" in the data the whole time, it just needed to be teased out.

In fairness, we can note that other regressors manage to derive the synthetic feature through their algorithmic structure.  But these regressors will have their own "blind spots" also, relative to different datasets.

In [ ]:
linf = pd.read_csv('data/linear_failure.csv', index_col=0)
X = linf.drop('TARGET', axis=1)
y = linf['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
dtr.score(X_test, y_test)

In [ ]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(X_train, y_train)
svr.score(X_test, y_test)

## Dimensionality reduction

There are two general approaches to *reducing* the number of dimensions (i.e. features) in a dataset.  One is by creating synthetic features that globally combine the raw features.  Several different mathematical techniques are available for doing this.

The other means by which we might reduce dimensions is simply by discarding ones that seem to have little significance to the model.  Very often models actually perform *better* by removing features that are either purely independent of the target or are largely redundant with other features (i.e. highly correlated).  In every case, models can be trained *faster* with fewer dimensions.

PCA is the oldest and most widely used method for decomposition of dimensional information.  Other methods of decomposition are also provided by scikit-learn.  In broad concept they do something similar, but each shows strengths relative to different datasets; understanding the difference is a mixture of domain familiarity and trial-and-error.

### Miscellaneous decompositions

This lesson will not specifically discuss all of the decomposition classes available in `sklearn.decomposition`.  A complete list with brief descriptions from the [documentation](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition) is below.  The discussion and examples of reconstructed faces in the [user guide](http://scikit-learn.org/stable/modules/decomposition.html) is very useful.

| Class name          | Description
|---------------------|------------------------------------
| DictionaryLearning  | Dictionary learning
| FactorAnalysis      | Factor Analysis (FA)
| FastICA             | FastICA: a fast algorithm for Independent Component Analysis.
| IncrementalPCA      | Incremental principal components analysis (IPCA).
| KernelPCA           | Kernel Principal component analysis (KPCA)
| LatentDirichletAllocation | Latent Dirichlet Allocation with online variational Bayes algorithm
| MiniBatchDictionaryLearning | Mini-batch dictionary learning
| MiniBatchSparsePCA  | Mini-batch Sparse Principal Components Analysis
| NMF                 | Non-Negative Matrix Factorization (NMF)
| PCA                 | Principal component analysis (PCA)
| SparsePCA           | Sparse Principal Components Analysis (SparsePCA)
| SparseCoder         | Sparse coding
| TruncatedSVD        | Dimensionality reduction using truncated SVD (aka LSA).
| dict_learning       | Solves a dictionary learning matrix factorization problem.
| dict_learning_online| Solves a dictionary learning matrix factorization problem online.
| fastica             | Perform Fast Independent Component Analysis.
| sparse_encode       | Sparse coding


### Principal component analysis (PCA)

Principal component analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components. The transformation is defined in such a way that the first principal component accounts for as much of the variability in the data as possible, and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal to the preceding components.

Let us look at an example. We will choose the Wisconsin breast cancer dataset we worked with in a previous lesson.  Recall that it has 30 features measuring a variety of numeric medical diagnostic results.

First thing, we load the dataset and apply standard scaling to the features.  Later in this lesson we will talk more about scaling, and we have brushed on it in a few previous lessons.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

cancer = load_breast_cancer()
scaler = StandardScaler()
scaler.fit(cancer.data)
X_scaled = scaler.transform(cancer.data)

Perform a standard train/test split as we have with almost all models.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, cancer.target, random_state=1)

For pending comparison, let us remind ourselves of how the raw data preforms using two classifers: `LinearRegression` and `KNeighborsClassifier`.

In [ ]:
from sklearn.linear_model import LinearRegression
(LinearRegression()
    .fit(X_train, y_train)
    .score(X_test, y_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
(KNeighborsClassifier()
    .fit(X_train, y_train)
    .score(X_test, y_test))

Now we transform the parametric space of the features into just two dimensions that contain the maximum amount of information that **can be** represented in two dimensions.

In [ ]:
cancer.data.shape

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA(n_components=2)
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)
print("Original shape: %s" % str(X_scaled.shape))
print("Reduced shape: %s" % str(X_pca.shape))

In [ ]:
cancer.feature_names

Looking at a scatter plot, we can see that just two dimensions already get rather good differentiation visually.  Notice that these two components each represent an arbitrary combination of all the actual observational measurements in the dataset.  Therefore, they do not have any obvious English description other than "first component" and "second component."

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# plot 1st vs 2nd principal component, color by class
plt.figure(figsize=(8, 8))
plt.scatter(X_pca[:, 0], X_pca[:, 1], 
            c=cancer.target, 
            cmap=mglearn.tools.cm, s=10)
plt.gca().set_aspect("equal")
plt.xlabel("First principal component")
plt.ylabel("Second principal component");

With just two components retained, the linear regression performs a bit worse, but KNN (that is much stronger so far for this data, in any case), performs basically equally well as with all 30 dimensions.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, cancer.target, random_state=1)

In [ ]:
(LinearRegression()
    .fit(X_train, y_train)
    .score(X_test, y_test))

In [ ]:
(KNeighborsClassifier()
    .fit(X_train, y_train)
    .score(X_test, y_test))

We can see how the two PCA components are derived by linear combination of the original 30.

In [ ]:
print(pca.components_)

Or more visually (but only some original features shown for legibility):

In [ ]:
# Not all original features shown...
nfeat = 12
plt.figure(figsize=(20,15))
plt.matshow(pca.components_[:,:nfeat], cmap='viridis')
plt.yticks([0, 1], ["first component", "second component"])
plt.colorbar()
plt.xticks(range(nfeat),
           cancer.feature_names[:nfeat], rotation=90, ha='left');
plt.xlabel("pca_components_cancer");

### Rotation, sphering and whitening

This course will not have time to address it, but additional preprocessing steps such as subtracting off the mean or calculating a z-score `(X-X.mean())/X.std()` can improve the fit of a PCA model.  The `PCA` class has options for some of these operations built in.

### Non-negative matrix factorization (NMF)

NMF finds two non-negative matrices whose product approximates the non-negative matrix X. This factorization can be used for example for dimensionality reduction, source separation or topic extraction.

In [ ]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=2)
nmf.fit(cancer.data)
X_nmf = nmf.transform(cancer.data)
print("Original shape: %s" % str(cancer.data.shape))
print("Reduced shape: %s" % str(X_pca.shape))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# plot 1st vs 2nd principal component, color by class
plt.figure(figsize=(6, 6))
plt.scatter(X_nmf[:, 0], X_nmf[:, 1], 
            c=cancer.target, 
            cmap=mglearn.tools.cm, s=10)
plt.gca().set_aspect("equal")
plt.xlabel("First NMF component")
plt.ylabel("Second NMF component");

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_nmf, cancer.target, random_state=1)
print(LinearRegression()
    .fit(X_train, y_train)
    .score(X_test, y_test))
print((KNeighborsClassifier()
    .fit(X_train, y_train)
    .score(X_test, y_test)))

### Latent Dirichlet Allocation (LDA)

Latent Dirichlet Allocation is a generative probabilistic model for collections of discrete dataset such as text corpora. It is also a topic model that is used for discovering abstract topics from a collection of documents.

This Bayesian technique has proven very useful for identifing hidden unobserved features in textual data sources.  It is not particularly well suited to the kind of numeric data we see in the cancer dataset, but we use that simply to show API usage here. Basically, it is the same as other decomposition classes—and generally the same as all the models throughout scikit-learn.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=2)
lda.fit(cancer.data)
X_lda = lda.transform(cancer.data)
print("Original shape: %s" % str(cancer.data.shape))
print("Reduced shape: %s" % str(X_lda.shape))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# plot 1st vs 2nd principal component, color by class
plt.figure(figsize=(6, 6))
plt.scatter(X_lda[:, 0], X_lda[:, 1], 
            c=cancer.target, 
            cmap=mglearn.tools.cm, s=10)
plt.gca().set_aspect("equal")
plt.xlabel("First LDA component")
plt.ylabel("Second LDA component");

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_lda, cancer.target, random_state=1)
print(LinearRegression()
    .fit(X_train, y_train)
    .score(X_test, y_test))
print((KNeighborsClassifier()
    .fit(X_train, y_train)
    .score(X_test, y_test)))

### Independent component analysis (ICA)

Independent component analysis separates a multivariate signal into additive subcomponents that are maximally independent. It is implemented in scikit-learn using the Fast ICA algorithm. Typically, ICA is not used for reducing dimensionality but for separating superimposed signals. 

> In PCA the basis you want to find is the one that best explains the variability of your data. The first vector of the PCA basis is the one that best explains the variability of your data (the principal direction) the second vector is the 2nd best explanation and must be orthogonal to the first one.

> In ICA the basis you want to find is the one in which each vector is an independent component of your data, you can think of your data as a mix of signals and then the ICA basis will have a vector for each independent signal.

> In a more practical way we can say that PCA helps when you want to find a reduced-rank representation of your data and ICA helps when you want to find a representation of your data as independent sub-elements. In layman terms PCA helps to compress data and ICA helps to separate data.

Quote credit: [Luis Argerich](https://www.quora.com/What-is-the-difference-between-PCA-and-ICA), Data Science Professor at the University of Buenos Aires (UBA)

In [ ]:
from sklearn.decomposition import FastICA
ica = FastICA(n_components=2)
ica.fit(cancer.data)
X_ica = ica.transform(cancer.data)
print("Original shape: %s" % str(cancer.data.shape))
print("Reduced shape: %s" % str(X_ica.shape))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# plot 1st vs 2nd principal component, color by class
plt.figure(figsize=(6, 6))
plt.scatter(X_ica[:, 0], X_ica[:, 1], 
            c=cancer.target, 
            cmap=mglearn.tools.cm, s=10)
plt.gca().set_aspect("equal")
plt.xlabel("First ICA component")
plt.ylabel("Second ICA component");

One might hope that since ICA is often used for separation of "signals" in noisy data, the cancer data might do better than PCA at detecting the "benign" and "malignant" signals that might be reflected in the indirect medical measurements.

At least without other parameterization, or other models, and looking at exactly two components, ICA does moderately worse than PCA.  Only very slightly so though; it is possible that other feature engineering might allow ICA to emerge as a good technicque for this problem area. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_ica, cancer.target, random_state=1)
print(LinearRegression()
    .fit(X_train, y_train)
    .score(X_test, y_test))
print((KNeighborsClassifier()
    .fit(X_train, y_train)
    .score(X_test, y_test)))

### t-distributed Stochastic Neighbor Embedding.

t-SNE is a tool to visualize high-dimensional data. It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. t-SNE has a cost function that is not convex, i.e. with different initializations we can get different results.

That is to say, t-SNE can often help models in similar ways to what PCA, NMF, LDA, or ICA do.  But it also is especially useful to create low-dimensional representations (i.e. two dimensions can actually fit on your computer monitors or printed books).

An example that shows t-SNE well is the digit dataset that we saw in an earlier lesson.  Let us load that data, and decompose it with both PCA and t-SNE.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

fig, axes = plt.subplots(2, 5, figsize=(10, 5),
                         subplot_kw={'xticks':(), 'yticks': ()})
for ax, img in zip(axes.ravel(), digits.images):
    ax.imshow(img, cmap=plt.get_cmap('Greys'))

In [ ]:
# build a PCA model
pca = PCA(n_components=2)
pca.fit(digits.data)

# transform the digits data onto the first two principal components
digits_pca = pca.transform(digits.data)
colors = ["#476A2A", "#7851B8", "#BD3430", "#4A2D4E", "#875525",
          "#A83683", "#4E655E", "#853541", "#3A3120","#535D8E"]
plt.figure(figsize=(10, 10))
plt.xlim(digits_pca[:, 0].min(), digits_pca[:, 0].max())
plt.ylim(digits_pca[:, 1].min(), digits_pca[:, 1].max())
for i in range(len(digits.data)):
    # actually plot the digits as text instead of using scatter
    plt.text(digits_pca[i, 0], digits_pca[i, 1], str(digits.target[i]),
             color = colors[digits.target[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("first principal component")
plt.ylabel("second principal component");

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42)
# use fit_transform instead of fit, as TSNE has no transform method:
digits_tsne = tsne.fit_transform(digits.data)

In [ ]:
plt.figure(figsize=(10, 10))
plt.xlim(digits_tsne[:, 0].min(), digits_tsne[:, 0].max() + 1)
plt.ylim(digits_tsne[:, 1].min(), digits_tsne[:, 1].max() + 1)
for i in range(len(digits.data)):
    # actually plot the digits as text instead of using scatter
    plt.text(digits_tsne[i, 0], digits_tsne[i, 1], str(digits.target[i]),
             color = colors[digits.target[i]],
             fontdict={'weight': 'bold', 'size': 9})

## Feature Selection

One way to reduce features in a dataset is to throw some of them away.  In principle, features thrown away can be engineered features such as those resulting from decompositions or those discussed below that are combined or synthesized from raw features.  Scikit-learn gives you several ways to choose features to discard; or euqivalently, features to keep.

There are several univariate feature selectors in scikit-learn.  `SelectKBest` is essentially the same as `SelectPercentile`, merely different in whether you indicate a number of percentage of features to keep.  Obviously, it is easy to derive one from the other based on number of features in the raw data. Just slightly different are false positive rate (`SelectFpr`), false discovery rate (`SelectFdr`), or family wise error (`SelectFwe`).  

All of these let you specify a scoring function the judges the quality of the estimate using just one feature.  For regression, there are `f_regression`, `mutual_info_regression`; for classification: `chi2`, `f_classif`, `mutual_info_classif`.  You can also, in principle, use a custom function to evaluate strength of a single feature for prediction.

### Univariate

Let us look at the Wisconsin cancer dataset from perspective of simply identifying the most important features (in the raw dataset).  Notice that we use the entire dataset for this purpose, not only a training portion of it.

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selector = SelectKBest(chi2, k=5)
X_new = selector.fit_transform(cancer.data, cancer.target)
cancer.data.shape, X_new.shape

In [ ]:
kbest = pd.Series(selector.scores_, index=cancer.feature_names).sort_values(ascending=False)
kbest.head(10)

Let us take a look at how models work with all 30 features versus with just the top 5 in a univariate filter.  In this example, we do not get better with this step alone (but KNN doesn't get much worse), but in some cases the score will actually improve without surpul dimensions.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, random_state=1)
print(LinearRegression()
    .fit(X_train, y_train)
    .score(X_test, y_test))
print((KNeighborsClassifier()
    .fit(X_train, y_train)
    .score(X_test, y_test)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_new, cancer.target, random_state=1)
print(LinearRegression()
    .fit(X_train, y_train)
    .score(X_test, y_test))
print((KNeighborsClassifier()
    .fit(X_train, y_train)
    .score(X_test, y_test)))

### Model-based Feature Selection

A number of selectors are based on the chosen underlying model itself rather on univariate strengths of features.  This only works with models than expose a `.coef_` or `.feature_importances_` attribute on a trained model.  Not all algorithms allow the determination of those values.

We look here at recursive feature elimination which is based on a model, and arrives at the most important features by eliminating the least important, then refitting based on those that remain, and doing so repeatedly until only the number of features requested remain.  The selector `SelectFromModel` works in a similar way, but recursive selection will generally be better quality (albeit slower since it involves repeated refitting).

In [ ]:
# Recursive feature elimination
from sklearn.feature_selection import RFE

lr = LinearRegression()
rfe = RFE(estimator=lr, n_features_to_select=5, step=1)
rfe.fit(cancer.data, cancer.target)

pd.Series(rfe.ranking_, index=cancer.feature_names).sort_values(ascending=False).head(10)

The relative importance of features is similar, but somewhat different under recursive feature elimination than under k-best selection.  Given the nature of linear models, we mostly expect univariate elimination to behave similarly to recursive elimination based on linear regression.  Something like a decision tree (which provides `.feature_importances_`) might have a more different ranking of selected features.

We can go farther here and use `RFECV` to recursive eliminate features, cross-validate the model at each stage, and keep a record of the relationship between model score and number of features.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV

rf = RandomForestClassifier(max_depth=7, random_state=1)
rfecv = RFECV(estimator=rf)
rfecv.fit(cancer.data, cancer.target)

# Plot number of features VS. cross-validation scores
plt.figure(figsize=(10,5))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.title("Optimal number of features : %d" % rfecv.n_features_)
plt.show()

## Dimensionality Expansion

There are two standard ways in which you are likely to engineer new synthetic features based on existing features: polynomial features and one-hot encoding.  In a sense, the decompositions also do the same thing—but they create synthetic features globally across the parametric space, and they generally are used as replacements rather than supplements to raw features.

### Polynomial Features

Generating polynomial features will create a very large number of new features.  The basic idea is simple, we add new features that are the multiplicative product of up to degree=N of the existing features.  In the toy example at the beginning of this lesson, we manually synthesized one feature by multiplying two existing ones together.  The `PolyFeatures` construction does so with all combinations of parameters.

Often using polynomial features is a large part of the reason it is particularly important to go back and winnow features using feature selection.  Reducing 30 features to 19 in the above example is unlikely to be hugely important to most models.  But reducing the 496 synthetic features in the below example becomes important (let alone the much larger number if you choose a higher degree or started with more raw features).

If the `interactions_only` option is not used, the number of produced features is:

$$ \#Features = N + N + \frac{N \times (N-1)}{2} + 1 $$

E.g. for degree=30, it is 496; for degree=100, it is 5151.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(cancer.data)
cancer.data.shape, X_poly.shape

In [ ]:
poly_names = poly.get_feature_names(cancer.feature_names)
pd.DataFrame(X_poly, columns=poly_names).head()

In [ ]:
rfc = RandomForestClassifier(max_depth=7, random_state=1)
rfe = RFE(estimator=rfc, n_features_to_select=50, step=1)

X_poly_top = rfe.fit_transform(X_poly, cancer.target)
X_poly_top.shape

In [ ]:
pd.Series(rfe.ranking_, index=poly_names).sort_values(ascending=False).head(15)

In [ ]:
# Scale the engineered features (makes little difference for this model, but is good practice)
X_poly_top_scaled = StandardScaler().fit_transform(X_poly)

X_train, X_test, y_train, y_test = train_test_split(
    X_poly_top_scaled, cancer.target, random_state=42)

rfc.fit(X_train, y_train).score(X_test, y_test)

### One-Hot Encoding

We have looked in previous lessons at the need to encode categorical values in **one-hot encoding**.  That is, we might have one feature with a a number of class values encoded in it.  For many models, this is either better quality—or simply required for the code to operate—than trying to use the class labels.  In some cases, integer values might work algorithmically, but will distort the training by being interpreted in a quantitative or ordinal way.

The interfaces provided by scikit-learn are servicable, but somewhat awkward.  `sklearn.preprocessing.LabelBinarizer` does almost what you want in some cases, but doesn't expose the clearest API.  The same can be said of `sklearn.preprocessing.OneHotEncoder` and `sklearn.preprocessing.LabelEncoder` and a couple others.  I simply recommend using `pandas.get_dummies()` in place of these others.  The result will be the same, in any case.

Let us look at a small toy example with catgorical data.

In [ ]:
pets = pd.read_csv('data/pets.csv')
pets

In [ ]:
pd.get_dummies(pets)

Performing the same encoding using the native scikit-learn classes and methods is a little bit more work.  `OneHotEncoder` chooses somewhat different column names that are a bit less descriptive.  There is not a straightforward way to specify the desciptive parts of the original column names rather than `x0`, `x1` etc.  Also, if you have a mixture of categorical and quantitative columns, specifying that is cumbersome compared to Pandas simply auto-detecting it for you.

On the other hand, if you do not wish to use Pandas, everything in scikit-learn happily works with the underlying NumPy arrays alone.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(pets)
one_hot_pets = enc.transform(pets)
columns = enc.get_feature_names()
pd.DataFrame(one_hot_pets.toarray(), columns=columns)

## Scaling

We have seen at several places the passing use of scaling to make data better regularized for models to perform best.  Scikit-learn provides several scaler classes that follow a similar API as models and other feature transformations.  Using these is mostly a simple drop-in step.

To make examples simple, we generate a small amount of random data with values of "features" in somewhat different ranges and distributions.

In [ ]:
import numpy as np
import numpy.random as npr

rows = 100
test_data = np.empty((rows,4))
test_data[:,0] = npr.random(rows) * 2 + 1.5 
test_data[:,1] = npr.randn(rows)
test_data[:,2] = npr.randint(-50, 25, rows)/10
test_data[:,3] = np.exp(npr.random(rows)+1.5)
print(test_data[:5])

In [ ]:
pd.DataFrame(test_data).describe().head(3)

In [ ]:
pd.DataFrame(test_data).plot(kind='kde');

In [ ]:
pd.DataFrame(test_data).plot(kind='hist', bins=300);

### StandardScaler

The most commonly used scaler—as the name indicates—is `StandardScaler`. 

This standardizes features by removing the mean and scaling to unit variance.  It loosely assumes that the underlying data is Gaussian to start with, but mostly it is fairly robust against moderate violations of that distribution.  Calculating the mean and standard deviation is per-column (as for all scalers).

In [ ]:
from sklearn.preprocessing import StandardScaler
scaled1 = StandardScaler().fit_transform(test_data)
scaled1[:5]

In [ ]:
pd.DataFrame(scaled1).describe().head(3)

In [ ]:
pd.DataFrame(scaled1).plot(kind='kde');

In [ ]:
pd.DataFrame(scaled1).plot(kind='hist', bins=300);

### RobustScaler

This scales features using statistics that are robust to outliers. It removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).

In [ ]:
from sklearn.preprocessing import RobustScaler
scaled2 = RobustScaler().fit_transform(test_data)
scaled2[:5]

In [ ]:
pd.DataFrame(scaled2).describe().head(3)

In [ ]:
pd.DataFrame(scaled2).plot(kind='kde');

In [ ]:
pd.DataFrame(scaled2).plot(kind='hist', bins=300);

### MinMaxScaler

Transforms features by scaling each feature to a given range. This estimator scales and translates each feature individually such that it is in the given range on the training set, by default between zero and one.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaled3 = MinMaxScaler().fit_transform(test_data)
scaled3[:5]

In [ ]:
pd.DataFrame(scaled3).describe()

In [ ]:
pd.DataFrame(scaled3).plot(kind='kde');

In [ ]:
pd.DataFrame(scaled3).plot(kind='hist', bins=300);

### MaxAbsScaler

Scales each feature by its maximum absolute value.  This estimator scales and translates each feature individually such that the maximal absolute value of each feature in the training set will be 1.0. It does not shift/center the data, and thus does not destroy any sparsity.

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
scaled4 = MaxAbsScaler().fit_transform(test_data)
scaled4[:5]

In [ ]:
pd.DataFrame(scaled4).describe()

In [ ]:
pd.DataFrame(scaled4).plot(kind='kde');

In [ ]:
pd.DataFrame(scaled4).plot(kind='hist', bins=300);

### ...and others

A number of other scalers exists for special purposes.  An interesting one is `QuantileTransformer` that normalizes feature values by quantile (default `n_quantiles=1000`).  In effect this treats values in an ordinal way, eliminating any effect of outliers.  Output distributions other than the default `'uniform'` can be selected as well.

# Binning values binarize

In the very first example in this course, we manually binarized a target.  We decided that one of those 1-10 scale ratings would have a cut-off point for "success" versus "failure."  That sort of thing is perfectly easy to construct using Pandas predicate filters (or similarly in NumPy).  But `sklearn.preprocessing.Binarizer` is available to accomplish the same end.

In [ ]:
data = pd.read_csv('data/Learning about Humans learning ML.csv')
target = data[['How successful has this tutorial been so far?']]
target.head()

In [ ]:
(target >=8).head()

In [ ]:
from sklearn.preprocessing import Binarizer
binary = Binarizer(threshold=7.5).fit_transform(target)
binary[:5]

In [ ]:
pd.DataFrame(binary).describe()

In [ ]:
pd.DataFrame(binary).plot();

## Next lesson

**Pipelines**: In this lesson we dealt with quite a lot of what we might call *data engineering*. The need for feature selection, feature engineering, and scaling, goes beyond the base fact that "data always comes in messy."  Certainly the need for data cleaning is important, and has probably not been addressed to match that importance in this course.  But even data that is spotless for an data integrity and anomaly perspective (a lofty goal) might still need to be massaged to allow models to succeed in their magic.

The next lesson on pipelines will be fairly short, but very important.  We have seen a variety of ways we can manipulate data—and also ways we can tune models with hyperparameters—but we have done each of these steps one-by-one, in explicit code with extra intermediate steps lying around along the way.  Pipelines allow us to package all these steps together in one resusable API.

<a href="Pipelines.ipynb"><img src="img/open-notebook.png" align="left"/></a>